In [46]:
import pandas as pd
import pyarrow.feather as feather
import numpy as np
from uncertainties import ufloat

In [8]:
era='2017'
import os 
os.listdir('data')

df = feather.read_feather('data/combined_{}.feather'.format(era))

In [25]:
df_bck = df[df.type=='bck']
names = {'DY':['ZToEE_M_120_200', 'ZToEE_M_200_400', 'ZToEE_M_400_800',
       'ZToEE_M_50_120', 'ZToEE_M_800_1400', 'ZToMuMu_M_120_200',
       'ZToMuMu_M_200_400', 'ZToMuMu_M_400_800', 'ZToMuMu_M_50_120',
       'ZToMuMu_M_800_1400'],
        'TT':['mc_ttbar'],
        'ST':['mc_santitop', 'mc_stop'],
        'VB':['mc_ww', 'mc_wz', 'mc_zz']}

In [17]:
regions = ['SR{}_nom', 'CR{}3_nom','CR{}0_nom','CR{}4_nom']
nJets = [1,2]

In [49]:
name_dict = {}
for name, namelist in names.items():
    region_dict = {}
    for nJet in nJets:  
        for reg in regions:
            reg = reg.format(nJet)
            sum_val = df_bck[(df_bck[reg]==1) & (df_bck.name.isin(namelist))]['Weight'].sum().round()
            sumw2 =  df_bck[(df_bck[reg]==1) & (df_bck.name.isin(namelist))]['Weight'].to_numpy()
            region_dict[reg] = ufloat(sum_val,np.sum(sumw2**2)**.5)
    name_dict[name] = region_dict

In [51]:
df = pd.DataFrame(name_dict)

In [52]:
df

,DY,TT,ST,VB
SR1_nom,(2.63+/-0.21)e+03,(2.82+/-0.05)e+03,604+/-11,16.0+/-2.3
CR13_nom,(1.82+/-0.15)e+03,(2.13+/-0.05)e+03,469+/-10,16.0+/-2.4
CR10_nom,(4.94+/-0.08)e+04,(1.47+/-0.04)e+03,334+/-9,247+/-10
CR14_nom,(3.94+/-0.07)e+04,(1.20+/-0.04)e+03,266+/-8,212+/-9
SR2_nom,(7.4+/-1.0)e+02,(4.40+/-0.07)e+03,285+/-8,22.0+/-2.8
CR23_nom,(7.0+/-1.2)e+02,(3.44+/-0.06)e+03,217+/-7,14.0+/-2.2
CR20_nom,(8.4+/-0.4)e+03,854+/-32,85+/-4,112+/-7
CR24_nom,(7.1+/-0.4)e+03,639+/-28,70+/-4,96+/-7


In [54]:
cr_sums = df.sum(axis=1)

In [56]:
df.divide(cr_sums, axis=0)

,DY,TT,ST,VB
SR1_nom,0.433+/-0.020,0.465+/-0.017,0.099+/-0.004,0.0026+/-0.0004
CR13_nom,0.410+/-0.020,0.481+/-0.017,0.106+/-0.004,0.0036+/-0.0005
CR10_nom,0.9601+/-0.0010,0.0286+/-0.0009,0.00649+/-0.00020,0.00480+/-0.00020
CR14_nom,0.9590+/-0.0012,0.0293+/-0.0010,0.00648+/-0.00023,0.00517+/-0.00024
SR2_nom,0.135+/-0.016,0.808+/-0.015,0.0523+/-0.0018,0.0040+/-0.0005
CR23_nom,0.159+/-0.023,0.788+/-0.022,0.0497+/-0.0021,0.0032+/-0.0005
CR20_nom,0.889+/-0.006,0.090+/-0.005,0.0090+/-0.0006,0.0119+/-0.0009
CR24_nom,0.898+/-0.006,0.081+/-0.005,0.0088+/-0.0007,0.0121+/-0.0010


In [62]:
print(df.divide(cr_sums, axis=0).to_latex().replace("+/-","$\pm$"))

\begin{tabular}{lllll}
\toprule
{} &               DY &               TT &                 ST &                 VB \\
\midrule
SR1\_nom  &    0.433$\pm$0.020 &    0.465$\pm$0.017 &      0.099$\pm$0.004 &    0.0026$\pm$0.0004 \\
CR13\_nom &    0.410$\pm$0.020 &    0.481$\pm$0.017 &      0.106$\pm$0.004 &    0.0036$\pm$0.0005 \\
CR10\_nom &  0.9601$\pm$0.0010 &  0.0286$\pm$0.0009 &  0.00649$\pm$0.00020 &  0.00480$\pm$0.00020 \\
CR14\_nom &  0.9590$\pm$0.0012 &  0.0293$\pm$0.0010 &  0.00648$\pm$0.00023 &  0.00517$\pm$0.00024 \\
SR2\_nom  &    0.135$\pm$0.016 &    0.808$\pm$0.015 &    0.0523$\pm$0.0018 &    0.0040$\pm$0.0005 \\
CR23\_nom &    0.159$\pm$0.023 &    0.788$\pm$0.022 &    0.0497$\pm$0.0021 &    0.0032$\pm$0.0005 \\
CR20\_nom &    0.889$\pm$0.006 &    0.090$\pm$0.005 &    0.0090$\pm$0.0006 &    0.0119$\pm$0.0009 \\
CR24\_nom &    0.898$\pm$0.006 &    0.081$\pm$0.005 &    0.0088$\pm$0.0007 &    0.0121$\pm$0.0010 \\
\bottomrule
\end{tabular}



In [63]:
print(df.to_latex().replace("+/-","$\pm$"))

\begin{tabular}{lllll}
\toprule
{} &                 DY &                 TT &        ST &          VB \\
\midrule
SR1\_nom  &  (2.63$\pm$0.21)e+03 &  (2.82$\pm$0.05)e+03 &  604$\pm$11 &  16.0$\pm$2.3 \\
CR13\_nom &  (1.82$\pm$0.15)e+03 &  (2.13$\pm$0.05)e+03 &  469$\pm$10 &  16.0$\pm$2.4 \\
CR10\_nom &  (4.94$\pm$0.08)e+04 &  (1.47$\pm$0.04)e+03 &   334$\pm$9 &    247$\pm$10 \\
CR14\_nom &  (3.94$\pm$0.07)e+04 &  (1.20$\pm$0.04)e+03 &   266$\pm$8 &     212$\pm$9 \\
SR2\_nom  &    (7.4$\pm$1.0)e+02 &  (4.40$\pm$0.07)e+03 &   285$\pm$8 &  22.0$\pm$2.8 \\
CR23\_nom &    (7.0$\pm$1.2)e+02 &  (3.44$\pm$0.06)e+03 &   217$\pm$7 &  14.0$\pm$2.2 \\
CR20\_nom &    (8.4$\pm$0.4)e+03 &           854$\pm$32 &    85$\pm$4 &     112$\pm$7 \\
CR24\_nom &    (7.1$\pm$0.4)e+03 &           639$\pm$28 &    70$\pm$4 &      96$\pm$7 \\
\bottomrule
\end{tabular}

